## Import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# For reproducibility
torch.manual_seed(1)

## Training Data

In [3]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [4]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [5]:
print(x_train.shape) # m = 6, d = 2
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


## Computiong the Hypothesis

$H(X)={1\over1+e^{-W^TX}} => {1\over1+e^{-XW}} => {1\over1+e^{-(XW+b)}}$

In [6]:
# PyTorch는 torch.exp()함수를 가지고 있다.
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [7]:
W = torch.zeros([2, 1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [8]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))
# x_train.matmul(W) = torch.matmul(X, W)

In [14]:
print(hypothesis)
print(hypothesis. shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


In [11]:
# PyTorch는 torch.sigmoid()함수도 가지고 있다.
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


좀 더 깔끔하게 hypothesis를 표현할 수 있다.

In [12]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [15]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


## Computing the Cost Function

$cost(W)=-{1\over m}\sum_{i=1}^m ylog(H(x)) + (1 - y)(log(1 - H(x))$

In [17]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


element하나에 대해서만 보면 loss는 다음과 같다.

In [18]:
# y_train[0]값과 1 - y_train[0]값 둘중 하나만 1로 살아남는다.
-(y_train[0] * torch.log(hypothesis[0]) + (1 - y_train[0]) * torch.log(1 - hypothesis[0]))

tensor([0.6931], grad_fn=<NegBackward>)

전체 sample에 대해서 구현해보면 loss는 다음과 같다.

In [20]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


수식의 맨앞부분 시그마 부분을 계산하기 위해 평균을 취한다.

In [21]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


 ##### 지금까지의 수식을 한 번에

In [22]:
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

## Whole Training Procedure

In [24]:
# 모델 초기화
W = torch.zeros([2, 1], requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## Evaluation

In [25]:
# 실제로는 x_train이 아니라 x_test로 해줘야함
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


확률을 구했다면 이제 0.5와 비교해서 0또는 1로 예측을 해야한다.

In [26]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [27]:
correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])


## Higher Implementation with Class

In [31]:
# logistic classification을 수행하는 모델 클래스 선언
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1) # W (2개의 element가진)
        self.sigmoid = nn.Sigmoid()   # b
        
    def forward(self, x):
        return self.sigmoid(self.linear(x)) # Wx + b

In [32]:
model = BinaryClassifier()

In [34]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    # cost로 H(x) 계산
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 10번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        # 정확성은 평균으로 구함
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/100 Cost: 0.897528 Accuracy 50.00%
Epoch   10/100 Cost: 0.546400 Accuracy 83.33%
Epoch   20/100 Cost: 0.588509 Accuracy 83.33%
Epoch   30/100 Cost: 0.478054 Accuracy 83.33%
Epoch   40/100 Cost: 0.392104 Accuracy 83.33%
Epoch   50/100 Cost: 0.311146 Accuracy 83.33%
Epoch   60/100 Cost: 0.238634 Accuracy 83.33%
Epoch   70/100 Cost: 0.184601 Accuracy 100.00%
Epoch   80/100 Cost: 0.156515 Accuracy 100.00%
Epoch   90/100 Cost: 0.143471 Accuracy 100.00%
Epoch  100/100 Cost: 0.133744 Accuracy 100.00%
